In [1]:
# Change working directory to one level higher within file system
%cd ..

/Users/kevinmildau/Dropbox/univie/Project embedding a molecular network/development/specXplore-prototype


In [2]:
%load_ext autoreload
%autoreload 2
from specxploreImporting import importing
import matchms
from matchms.importing import load_from_mgf
from matchms.exporting import save_as_mgf
from ms2query.run_ms2query import download_zenodo_files, run_complete_folder
from ms2query.ms2library import create_library_object_from_one_dir
import os
import pandas as pd
import numpy as np
from typing import List

In [3]:
models_and_library_folder_pos = os.path.join("data_and_output", "models", "pos")
raw_experimental_mgf_filename = os.path.join("data_and_output", "npl", "GNPS-NIH-NATURALPRODUCTSLIBRARY.mgf")
raw_standards_mgf_filename = os.path.join("data_and_output", "npl", "phophe_pos.mgf")
processed_experimental_spectra_filename = os.path.join("data_and_output", "npl", "spectra_experimental.mgf")
processed_standard_spectra_filename = os.path.join("data_and_output", "npl", "spectra_standards.mgf")
csv_experimental_filename = os.path.join("data_and_output", "npl", "nlp_ms2query_out_experimental.csv")
csv_standards_filename = os.path.join("data_and_output", "npl", "nlp_ms2query_out_standard.csv")
scores_s2v_filename = os.path.join("data_and_output", "npl", "s2v.npy")
scores_ms2ds_filename = os.path.join("data_and_output", "npl", "ms2ds.npy")
scores_cos_filename = os.path.join("data_and_output", "npl", "cos.npy")

gnps_graphml_filepath = os.path.join("data_and_output", "npl", "gnps_export", "ProteoSAFe-METABOLOMICS-SNETS-V2-b425bf40-download_cytoscape_data", "gnps_molecular_network_graphml", "7d442038cc24444f8d16dcb8ed8a9c6d.graphml")

# Process Raw Data


In [4]:
spectra_experimental = list(load_from_mgf(raw_experimental_mgf_filename))
spectra_experimental = importing.clean_spectra(spectra_experimental)


spectra_standards = list(load_from_mgf(raw_standards_mgf_filename))
spectra_standards = importing.clean_spectra(spectra_standards)
if False:
    save_as_mgf(spectra_standards, processed_standard_spectra_filename)
    save_as_mgf(spectra_experimental, processed_experimental_spectra_filename)

In [7]:
print(spectra_experimental[0].metadata)
print(spectra_experimental[0].metadata["spectrumid"])

{'pepmass': (408.156, None), 'charge': 1, 'mslevel': '2', 'source_instrument': 'LC-ESI-qTof', 'filename': 'p1-A05_GA5_01_17878.mzXML', 'seq': '*..*', 'ionmode': 'positive', 'organism': 'GNPS-NIH-NATURALPRODUCTSLIBRARY', 'name': '"MLS001141366-01!(2R)-2-[3-[(10aS)-1,3-dioxo-10,10a-dihydro-5H-imidazo[1,5-b]isoquinolin-2-yl]propanoylamino]-2-phenylacetic acid" M+H', 'pi': 'Dorrestein', 'datacollector': 'VVP/LMS', 'smiles': 'OC(=O)[C@H](NC(=O)CCN1C(=O)[C@@H]2Cc3ccccc3CN2C1=O)c4ccccc4', 'inchi': '', 'inchiaux': 'N/A', 'pubmed': 'N/A', 'submituser': 'vphelan', 'libraryquality': '1', 'spectrumid': 'CCMSLIB00000079350', 'scans': '1865', 'compound_name': '(2R)-2-[3-[(10aS)-1,3-dioxo-10,10a-dihydro-5H-imidazo[1,5-b]isoquinolin-2-yl]propanoylamino]-2-phenylacetic acid"', 'adduct': '[M+H]+', 'precursor_mz': 408.156, 'inchikey': ''}
CCMSLIB00000079350


# Run Classification using ms2query

In [24]:
# WARNING
# THIS DOES WORK, BUT AUTOMATICALLY RUNS BOTH STANDARDS AND EXPERIMENTAL SPECTRA IF IN SAME FOLDER
# UPDATE TO MS2QUERY LATEST VERSION TO ALLOW SINGLE FILE INPUT AND OUTPUT
# ONCE UPDATED, FILENAMES CAN ALSO BE SPECIFIED....
if False:
    ms2library = create_library_object_from_one_dir(models_and_library_folder_pos)
    importing.run_single_file(ms2library, processed_standard_spectra_filename, str(csv_standards_filename))
    importing.run_single_file(ms2library, processed_experimental_spectra_filename, str(csv_experimental_filename))

Predicting matches for query spectra: 100%|██████████| 1267/1267 [23:00<00:00,  1.09s/it]


# Merge Classes Tables and Spectra lists

In [5]:
classes_experimental = pd.read_csv(csv_experimental_filename)
classes_experimental = importing.expand_ms2query_results_table(classes_experimental, len(spectra_experimental))
classes_standards = pd.read_csv(csv_standards_filename)
classes_standards = importing.expand_ms2query_results_table(classes_standards, len(spectra_standards))

In [6]:
classes_experimental["is_standard"] = False
classes_experimental["exp_metadata"] = None
classes_standards["is_standard"] = True
classes_standards["standard_metadata"] = None

gnps_molfam_assignment_df = importing.extract_molecular_family_assignment_from_graphml(gnps_graphml_filepath)
gnps_molfam_assignment_df["source_spectrum_mgf_iloc"] = gnps_molfam_assignment_df['idx']
gnps_molfam_assignment_df.drop(['id', 'spectrum_id', 'idx'], axis=1, inplace = True)
classes_experimental = classes_experimental.merge(gnps_molfam_assignment_df, on =["source_spectrum_mgf_iloc"], how = "left")


all_class_table = pd.merge(classes_experimental, classes_standards, how = 'outer').reset_index()
all_class_table["specxplore_id"] = all_class_table.index
all_class_table

,index,query_spectrum_nr,ms2query_model_prediction,precursor_mz_difference,precursor_mz_query_spectrum,precursor_mz_analog,inchikey,spectrum_ids,analog_compound_name,retention_time,...,cf_direct_parent,npc_class_results,npc_superclass_results,npc_pathway_results,source_spectrum_mgf_iloc,is_standard,exp_metadata,molecular_family,standard_metadata,specxplore_id
0,0,1,0.5991,0.0000,408.1560,408.156,XTJNPXYDTGJZSA,11275,"(2R)-2-[3-[(10aS)-1,3-dioxo-10,10a-dihydro-5H-...",NaN,...,Hybrid peptides,NaN,NaN,Amino acids and Peptides,0,False,None,51,NaN,0
1,1,2,0.8834,0.0000,277.0980,277.098,VOYWJNWCKFCMPN,11276,"(2S)-3-benzoyl-2,7b-dihydro-1aH-oxireno[2,3-c]...",NaN,...,Hydroquinolines,NaN,NaN,NaN,1,False,None,-1,NaN,1
2,2,3,0.8556,0.0010,269.0810,269.080,IRZVHDLBAYNPCT,129041,7-O-Methylchrysin,NaN,...,7-O-methylated flavonoids,Flavones,Flavonoids,Shikimates and Phenylpropanoids,2,False,None,54,NaN,2
3,3,4,0.6125,0.0000,492.1770,492.177,OPWCHZIQXUKNMP,11278,"N-[2-[7-methoxy-6-[(Z)-(2,4,6-trioxo-1-prop-2-...",NaN,...,Benzodioxoles,NaN,NaN,NaN,3,False,None,42,NaN,3
4,4,5,0.8579,0.0000,481.2450,481.245,GTBYYVAKXYVRHX,11675,(2R)-2-[[(2S)-3-methyl-2-[[(2S)-3-oxo-2-propan...,NaN,...,Dipeptides,NaN,Oligopeptides,Amino acids and Peptides,4,False,None,15,NaN,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1287,1287,21,0.4887,95.0433,579.1483,484.105,CNURKUNYCXCBEK,283054,karakin,NaN,...,Tricarboxylic acids and derivatives,NaN,NaN,NaN,20,True,NaN,NaN,NaN,1287
1288,1288,22,0.9419,0.0004,579.1494,579.149,XFZJEEAOWLFHDH,24733,Procyanidin B2,NaN,...,Biflavonoids and polyflavonoids,Proanthocyanins,Flavonoids,Shikimates and Phenylpropanoids,21,True,NaN,NaN,NaN,1288
1289,1289,23,0.9387,0.0002,581.1858,581.186,HXTFHSYLYXVTHC,13249,"5-hydroxy-2-(4-hydroxyphenyl)-7-[(2S,3R,4S,5S,...",NaN,...,Flavonoid-7-O-glycosides,Flavanones,Flavonoids,Shikimates and Phenylpropanoids,22,True,NaN,NaN,NaN,1289
1290,1290,24,0.9387,0.0002,611.1958,611.196,ARGKVCXINMKCAZ,18761,"(2S)-7-[(2S,3R,4S,5S,6R)-4,5-dihydroxy-6-(hydr...",NaN,...,Flavonoid-7-O-glycosides,Flavanones,Flavonoids,Shikimates and Phenylpropanoids,23,True,NaN,NaN,NaN,1290


In [7]:
all_spectra = spectra_experimental + spectra_standards # list addition

In [28]:
if False:
    scores_s2v = importing.compute_similarities_s2v(all_spectra, models_and_library_folder_pos)
    scores_cos = importing.compute_similarities_cosine(all_spectra, cosine_type="ModifiedCosine")
    scores_ms2ds = importing.compute_similarities_ms2ds(all_spectra, models_and_library_folder_pos)
    np.save(scores_ms2ds_filename, scores_ms2ds, allow_pickle=False)
    np.save(scores_s2v_filename, scores_s2v, allow_pickle=False)
    np.save(scores_cos_filename, scores_cos, allow_pickle=False)

Calculating vectors of reference spectrums: 100%|██████████| 1292/1292 [00:29<00:00, 43.20it/s]


In [16]:
scores_s2v = np.load(scores_s2v_filename)
scores_ms2ds = np.load(scores_ms2ds_filename)
scores_cos = np.load(scores_cos_filename)


scores_heuristic = np.maximum(scores_ms2ds, scores_cos)
scores_ms2ds = scores_heuristic # <-- overwriting ms2ds score here!!!

# Run Kmedoid and tsne

In [17]:
k_values = [10,20,30,40,50,75,100, 125, 150, 175, 200,250, 300, 400,500]
random_seeds = [int(np.random.randint(0,1000)) for _ in k_values]
similarities = scores_ms2ds
distances = importing.convert_similarity_to_distance(similarities)
kmedoid_list = importing.run_kmedoid_grid(distances, k_values, random_seeds)
importing.render_kmedoid_fitting_results_in_browser(kmedoid_list)

In [18]:
perplexity_values = [x for x in [30, 40, 50, 60, 100, 200]]
random_seeds = [int(np.random.randint(0,1000)) for _ in perplexity_values]
tsne_list_pos = importing.run_tsne_grid(distances, perplexity_values, random_seeds)
importing.render_tsne_fitting_results_in_browser(tsne_list_pos)

# Merge into specxplore object

In [19]:
from specxplore.specxplore_data import specxplore_data
is_standard = np.array(all_class_table["is_standard"])
spec_classes = all_class_table[['cf_kingdom', 'cf_superclass', 'cf_class', 'cf_subclass',
       'cf_direct_parent', 'npc_class_results', 'npc_superclass_results',
       'npc_pathway_results', 'molecular_family']]

mz = [spec.get("precursor_mz") for spec in all_spectra]
# KmedoidGridEntry(k, cluster_assignments, score, random_states[idx])
kclass_table = pd.DataFrame()
for elem in kmedoid_list:
    kclass_table[elem.k] = elem.cluster_assignments
classes = pd.concat([spec_classes, kclass_table], axis=1)

In [13]:



spectra_converted = importing.convert_matchms_spectra_to_specxplore_spectrum(all_spectra)

In [20]:
tsnedf = pd.DataFrame({"x" : tsne_list_pos[4].x_coordinates, "y" : tsne_list_pos[4].y_coordinates})
specxplore_id = np.array(all_class_table["specxplore_id"])

npl_specxplore = specxplore_data(
  scores_ms2ds,scores_s2v, scores_cos, tsnedf,  classes,
  is_standard, spectra_converted, mz, specxplore_id)

In [21]:
import pickle
with open('data_and_output/npl_out/npl_specxplore3-heuristic-ms2ds-cos-hybrid.pickle', 'wb') as file:
  pickle.dump(npl_specxplore, file)